# Movie Recommendation System

## Objective

The recommendation system analyzes the preferences of the user concerned, and then it uses this information to try to find similar movies.After that, the system provides movie recommendations for the user.

## Data Source


I access the dataset from Ybi Foundation Github Page

## Import Library

In [29]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

### Import Data

In [104]:
import pandas as pd

data = pd.read_csv('https://github.com/YBIFoundation/Dataset/raw/main/Movies%20Recommendation.csv',delimiter=',', encoding='utf-8')


In [105]:
data.head()

,Movie_ID,Movie_Title,Movie_Genre,Movie_Language,Movie_Budget,Movie_Popularity,Movie_Release_Date,Movie_Revenue,Movie_Runtime,Movie_Vote,...,Movie_Homepage,Movie_Keywords,Movie_Overview,Movie_Production_House,Movie_Production_Country,Movie_Spoken_Language,Movie_Tagline,Movie_Cast,Movie_Crew,Movie_Director
0,1,Four Rooms,Crime Comedy,en,4000000,22.876230,09-12-1995,4300000,98.0,6.5,...,NaN,hotel new year's eve witch bet hotel room,It's Ted the Bellhop's first night on the job....,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Twelve outrageous guests. Four scandalous requ...,Tim Roth Antonio Banderas Jennifer Beals Madon...,"[{'name': 'Allison Anders', 'gender': 1, 'depa...",Allison Anders
1,2,Star Wars,Adventure Action Science Fiction,en,11000000,126.393695,25-05-1977,775398007,121.0,8.1,...,http://www.starwars.com/films/star-wars-episod...,android galaxy hermit death star lightsaber,Princess Leia is captured and held hostage by ...,"[{""name"": ""Lucasfilm"", ""id"": 1}, {""name"": ""Twe...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","A long time ago in a galaxy far, far away...",Mark Hamill Harrison Ford Carrie Fisher Peter ...,"[{'name': 'George Lucas', 'gender': 2, 'depart...",George Lucas
2,3,Finding Nemo,Animation Family,en,94000000,85.688789,30-05-2003,940335536,100.0,7.6,...,http://movies.disney.com/finding-nemo,father son relationship harbor underwater fish...,"Nemo, an adventurous young clownfish, is unexp...","[{""name"": ""Pixar Animation Studios"", ""id"": 3}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","There are 3.7 trillion fish in the ocean, they...",Albert Brooks Ellen DeGeneres Alexander Gould ...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
3,4,Forrest Gump,Comedy Drama Romance,en,55000000,138.133331,06-07-1994,677945399,142.0,8.2,...,NaN,vietnam veteran hippie mentally disabled runni...,A man with a low IQ has accomplished great thi...,"[{""name"": ""Paramount Pictures"", ""id"": 4}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","The world will never be the same, once you've ...",Tom Hanks Robin Wright Gary Sinise Mykelti Wil...,"[{'name': 'Alan Silvestri', 'gender': 2, 'depa...",Robert Zemeckis
4,5,American Beauty,Drama,en,15000000,80.878605,15-09-1999,356296601,122.0,7.9,...,http://www.dreamworks.com/ab/,male nudity female nudity adultery midlife cri...,"Lester Burnham, a depressed suburban father in...","[{""name"": ""DreamWorks SKG"", ""id"": 27}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Look closer.,Kevin Spacey Annette Bening Thora Birch Wes Be...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes


## Describe Data

In [106]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4760 entries, 0 to 4759
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Movie_ID                  4760 non-null   int64  
 1   Movie_Title               4760 non-null   object 
 2   Movie_Genre               4760 non-null   object 
 3   Movie_Language            4760 non-null   object 
 4   Movie_Budget              4760 non-null   int64  
 5   Movie_Popularity          4760 non-null   float64
 6   Movie_Release_Date        4760 non-null   object 
 7   Movie_Revenue             4760 non-null   int64  
 8   Movie_Runtime             4758 non-null   float64
 9   Movie_Vote                4760 non-null   float64
 10  Movie_Vote_Count          4760 non-null   int64  
 11  Movie_Homepage            1699 non-null   object 
 12  Movie_Keywords            4373 non-null   object 
 13  Movie_Overview            4757 non-null   object 
 14  Movie_Pr

In [107]:
data.shape

(4760, 21)

In [108]:
data.columns

Index(['Movie_ID', 'Movie_Title', 'Movie_Genre', 'Movie_Language',
       'Movie_Budget', 'Movie_Popularity', 'Movie_Release_Date',
       'Movie_Revenue', 'Movie_Runtime', 'Movie_Vote', 'Movie_Vote_Count',
       'Movie_Homepage', 'Movie_Keywords', 'Movie_Overview',
       'Movie_Production_House', 'Movie_Production_Country',
       'Movie_Spoken_Language', 'Movie_Tagline', 'Movie_Cast', 'Movie_Crew',
       'Movie_Director'],
      dtype='object')

## Feature Selection

In [109]:
selected_features = data[['Movie_Genre', 'Movie_Keywords', 'Movie_Tagline',
       'Movie_Cast', 'Movie_Director']].fillna('')
print(selected_features)

                           Movie_Genre  \
0                         Crime Comedy   
1     Adventure Action Science Fiction   
2                     Animation Family   
3                 Comedy Drama Romance   
4                                Drama   
...                                ...   
4755                            Horror   
4756               Comedy Family Drama   
4757                    Thriller Drama   
4758                            Family   
4759                       Documentary   

                                         Movie_Keywords  \
0             hotel new year's eve witch bet hotel room   
1           android galaxy hermit death star lightsaber   
2     father son relationship harbor underwater fish...   
3     vietnam veteran hippie mentally disabled runni...   
4     male nudity female nudity adultery midlife cri...   
...                                                 ...   
4755                                                      
4756                   

In [110]:
# combining all the 5 selected features

combined_features = selected_features['Movie_Genre']+' '+selected_features['Movie_Keywords']+' '+selected_features['Movie_Tagline']+' '+selected_features['Movie_Cast']+' '+selected_features['Movie_Director']

In [111]:
print(combined_features)

0       Crime Comedy hotel new year's eve witch bet ho...
1       Adventure Action Science Fiction android galax...
2       Animation Family father son relationship harbo...
3       Comedy Drama Romance vietnam veteran hippie me...
4       Drama male nudity female nudity adultery midli...
                              ...                        
4755    Horror  The hot spot where Satan's waitin'. Li...
4756    Comedy Family Drama  It’s better to stand out ...
4757    Thriller Drama christian film sex trafficking ...
4758                                           Family    
4759    Documentary music actors legendary perfomer cl...
Length: 4760, dtype: object


## Feature Text Conversion To Tokens

In [112]:
vectorizer = TfidfVectorizer()

In [114]:
vect = vectorizer.fit_transform(combined_features)

## Similarity score using cosine similarity

In [115]:
similarity = cosine_similarity(vect)
print(similarity)

[[1.         0.01351235 0.03570468 ... 0.         0.         0.        ]
 [0.01351235 1.         0.00806674 ... 0.         0.         0.        ]
 [0.03570468 0.00806674 1.         ... 0.         0.08014876 0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.08014876 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [116]:
print(similarity.shape)

(4760, 4760)


## Get Movie name as input from user

In [117]:
#getting movie name from the user

movie_name = input("Enter your Movie : ")

Enter your Movie : Iron man


In [118]:
all_titles = data["Movie_Title"].tolist()
print(all_titles)

['Four Rooms', 'Star Wars', 'Finding Nemo', 'Forrest Gump', 'American Beauty', 'Dancer in the Dark', 'The Fifth Element', 'Metropolis', 'My Life Without Me', 'Pirates of the Caribbean: The Curse of the Black Pearl', 'Kill Bill: Vol. 1', 'Jarhead', 'Apocalypse Now', 'Unforgiven', 'The Simpsons Movie', 'Eternal Sunshine of the Spotless Mind', 'Amores perros', "Pirates of the Caribbean: Dead Man's Chest", 'A History of Violence', '2001: A Space Odyssey', '8 Mile', 'Absolute Power', 'Brazil', 'Walk the Line', 'Million Dollar Baby', 'Billy Elliot', 'American History X', 'War of the Worlds', 'Mars Attacks!', 'Before Sunrise', 'Memento', 'Blade Runner', 'Hero', 'Before Sunset', 'Open Water', 'Raiders of the Lost Ark', 'Indiana Jones and the Temple of Doom', 'Indiana Jones and the Last Crusade', 'Beverly Hills Cop', 'Armageddon', 'Beverly Hills Cop II', 'Gladiator', 'Lock, Stock and Two Smoking Barrels', 'Taxi Driver', 'Run Lola Run', 'Back to the Future', 'Predator', 'Snatch', 'Scarface', 'Pr

In [119]:
# finding the close match
close_match = difflib.get_close_matches(movie_name, all_titles)
print(close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [120]:
find_close_match = close_match[0]
print(find_close_match)

Iron Man


In [121]:
#finding the index
movie_index = data[data.Movie_Title == find_close_match]['Movie_ID'].values[0]
print(movie_index)

568


In [122]:
# getting a list of similar movies
sim_score = list(enumerate(similarity[movie_index]))
print(sim_score)

[(0, 0.026990942208242455), (1, 0.010950507698138944), (2, 0.018806392274173616), (3, 0.028913528217961625), (4, 0.0), (5, 0.0), (6, 0.045278184449287086), (7, 0.018586667790081918), (8, 0.0), (9, 0.04536669475996404), (10, 0.06945245379615324), (11, 0.008931072914346624), (12, 0.018500407537554785), (13, 0.0), (14, 0.00789196005214734), (15, 0.01449570874675743), (16, 0.0), (17, 0.044703222447363855), (18, 0.008269983194635585), (19, 0.020222904227665553), (20, 0.01873282400803726), (21, 0.0), (22, 0.006966263873994487), (23, 0.009065984808544987), (24, 0.02272842249229887), (25, 0.02733081208457935), (26, 0.0), (27, 0.007242460629470042), (28, 0.030203200017920463), (29, 0.014894235274266102), (30, 0.0), (31, 0.0), (32, 0.11981278942051383), (33, 0.0076219084873372185), (34, 0.009583388440146605), (35, 0.0416606444301921), (36, 0.017574944265830113), (37, 0.04275348186567463), (38, 0.025841364057983304), (39, 0.038877689068816626), (40, 0.013801714159467784), (41, 0.01364183638501605

In [123]:
len(sim_score)

4760

## Get Sort all movies based on Similarity score

In [124]:
# sorting the movies based on their similarity score
sorted_movie = sorted(sim_score, key = lambda x:x[1], reverse = True)
print(sorted_movie)

[(568, 1.0000000000000002), (570, 0.23774638753738936), (3724, 0.1834643245085129), (2361, 0.16436286018967577), (418, 0.16093391568075596), (1799, 0.16089450005652564), (1681, 0.15366093144614124), (1540, 0.14326199460383862), (3590, 0.14011656375639475), (682, 0.13792739122041112), (3459, 0.13763496549881873), (495, 0.13172799619425368), (1732, 0.13151019053607485), (1333, 0.13068374655395776), (1736, 0.12838464360565432), (1113, 0.12722643271405384), (4279, 0.12378550300043255), (1578, 0.1218507485725904), (4742, 0.12136086712670666), (32, 0.11981278942051383), (2113, 0.11937772926054144), (2055, 0.11793110203851845), (1788, 0.11512033774631325), (4589, 0.10981794830020203), (1948, 0.10926697883678246), (668, 0.10384678106617842), (2697, 0.10280361085864685), (904, 0.10232723973821155), (1844, 0.10201289204738176), (3457, 0.09941298667653828), (2810, 0.09750823594715402), (1784, 0.09669182663465674), (1495, 0.0961889423177075), (905, 0.0958935386885064), (644, 0.0951957080181505), (

In [128]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_movie:
  index = movie[0]
  title_from_index = data[data.index==index]['Movie_Title'].values[0]
  if (i<31):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Niagara
2 . Caravans
3 . My Week with Marilyn
4 . Brokeback Mountain
5 . Harry Brown
6 . Night of the Living Dead
7 . The Curse of Downers Grove
8 . The Boy Next Door
9 . Back to the Future
10 . The Juror
11 . Some Like It Hot
12 . Enough
13 . The Kentucky Fried Movie
14 . Eye for an Eye
15 . Welcome to the Sticks
16 . Alice Through the Looking Glass
17 . Superman III
18 . The Misfits
19 . Premium Rush
20 . Duel in the Sun
21 . Sabotage
22 . Small Soldiers
23 . All That Jazz
24 . Camping Sauvage
25 . The Raid
26 . Beyond the Black Rainbow
27 . To Kill a Mockingbird
28 . World Trade Center
29 . The Dark Knight Rises
30 . Tora! Tora! Tora!


## Movie Recomendation System

In [129]:
movie_name = input(' Enter your favourite movie name : ')

all_titles = data["Movie_Title"].tolist()

close_match = difflib.get_close_matches(movie_name, all_titles)

find_close_match = close_match[0]

movie_index = data[data.Movie_Title == find_close_match]['Movie_ID'].values[0]

sim_score = list(enumerate(similarity[movie_index]))

sorted_movie = sorted(sim_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_movie:
  index = movie[0]
  title_from_index = data[data.index==index]['Movie_Title'].values[0]
  if (i<31):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : Star war
Movies suggested for you : 

1 . Finding Nemo
2 . Shark Tale
3 . Big Fish
4 . John Carter
5 . The Muse
6 . Megamind
7 . Mallrats
8 . Spider-Man
9 . Happy Feet
10 . The Last Temptation of Christ
11 . Meet the Deedles
12 . The English Patient
13 . Death Sentence
14 . When Did You Last See Your Father?
15 . The Life Aquatic with Steve Zissou
16 . Flight of the Intruder
17 . El Mariachi
18 . Treading Water
19 . Silent Movie
20 . Bottle Rocket
21 . WALL·E
22 . The Outsiders
23 . He Got Game
24 . Ponyo
25 . American Dreamz
26 . Zoom
27 . Shooting Fish
28 . Bambi
29 . Dear Frankie
30 . Freaky Friday
